In [7]:
from tools.preprocess import LoadDataset

import pandas as pd
import numpy as np

from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import IsolationForest
from catboost import Pool,CatBoostClassifier

train_path = "../본선/train.csv"
test_path = "../본선/test.csv"
# valid_path = "../본선/valid.csv"

# do with train, test, valid
data = LoadDataset(train_path,
                   test_path)

print(data.train.x.shape)

(40506, 149)


In [8]:
data.nan_processing({"ALL": "DROP COLUMN"})
data.sampling("OFF")
data.scaling({"ALL": "MINMAX"})
# data.encoding({"Collect Date_Dam": "LABEL",
#                "Collect Date_Fill1": "LABEL",
#                "Collect Date_Fill2": "LABEL",
#                "Collect Date_AutoClave": "LABEL",
#                "OTHERS": "LABEL"})
# data.feature_engineering("PCA PER PROCESS", {"min_proba": 0.9999})

Missing Value Processing...

(before processing)The number of nan
- train: 38298
  1. Processing <ALL> Columns with 'DROP COLUMN'

Finish!
(after processing)The number of nan
- train: 0

Sampling with OFF...

(before sampling)Value count
target
0    38156
1     2350
Name: count, dtype: int64 

Finish!
(after sampling)Value count
target
0    38156
1     2350
Name: count, dtype: int64

Scaling our dataset...

  1. Scaling <ALL> Columns with 'MINMAX'

Finish! (the number of columns: 130)



In [ ]:
import xgboost as xgb

dtrain = xgb.DMatrix(data.train.x, label=data.train.y)

params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse'
}

model = xgb.train(params, dtrain, num_boost_round=100)
y_pred = model.predict(dtrain)


# 예측 오차 계산
residuals = np.abs(y - y_pred)

# 이상치 기준 설정 (예: 평균 + 3표준편차)
threshold = residuals.mean() + 3 * residuals.std()

# 이상치 탐지
anomalies = df_sorted[residuals > threshold]
normal_data = df_sorted[residuals <= threshold]

In [39]:
data1 = pd.read_csv("../본선/train.csv")
data2 = pd.read_excel('../본선/hand_data.xlsx')

In [34]:
data2

,day,start_time,end_time,Dam Thickness - 1time / day,Fill thickness - 1time / day,Fill cure energy (mJ) - Every break time,Fill cure power (mW) - Every break time,Line Temp. - Every break time,Line humidity - Every break time,OCR gap - Line#1 stage1,OCR gap - Line#1 stage2,OCR gap - Line#1 stage3,OCR gap - Line #2 stage1,OCR gap - Line #2 stage2,OCR gap - Line #2 stage3
0,240319,08:00:00,10:30:00,454.0,193.0,1728.0,2919.0,23.4,56.6,1.2,1.000,1.200,1.2,0.900,1.4
1,240319,10:45:00,12:00:00,454.0,193.0,1754.0,2962.0,24.0,50.9,1.2,1.000,1.200,1.2,0.900,1.4
2,240319,13:00:00,15:00:00,454.0,193.0,1708.0,2887.0,24.5,49.6,1.2,1.000,1.200,1.2,0.900,1.4
3,240319,15:15:00,18:00:00,454.0,193.0,1751.0,2946.0,24.7,53.4,1.2,1.000,1.200,1.2,1.200,1.4
4,240319,18:30:00,20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,240428,08:00:00,10:30:00,478.0,232.0,1642.0,2952.0,24.6,53.0,1.0,1.232,1.466,1.2,1.167,1.5
101,240428,10:45:00,12:00:00,478.0,232.0,1679.0,2942.0,24.8,52.7,1.0,1.232,1.466,1.2,1.167,1.5
102,240428,13:00:00,15:00:00,478.0,232.0,1552.0,2941.0,24.6,55.5,1.0,1.232,1.466,1.2,1.167,1.5
103,240428,15:15:00,18:00:00,478.0,232.0,1552.0,2934.0,24.3,54.6,1.0,1.232,1.466,1.2,1.167,1.5


In [44]:
data.train.get_cols(['Date'])

['Collect Date_Dam',
 'Collect Date_AutoClave',
 'Collect Date_Fill1',
 'Collect Date_Fill2']

In [49]:
pd.to_datetime(data.train.x['Collect Date_Dam'])

0       2024-04-25 11:10:00
1       2023-09-19 14:30:00
2       2024-03-05 09:30:00
3       2023-09-25 15:40:00
4       2023-06-27 13:20:00
                ...        
40501   2023-09-14 15:50:00
40502   2024-04-10 14:20:00
40503   2024-02-22 08:40:00
40504   2023-07-25 11:00:00
40505   2023-05-31 01:40:00
Name: Collect Date_Dam, Length: 40506, dtype: datetime64[ns]

In [51]:
import pandas as pd

# data와 data2를 사용한다고 가정합니다.

# 'day' 열을 datetime 형식으로 변환 (YYMMDD 형식에서 변환)
data2['day'] = pd.to_datetime(data2['day'], format='%y%m%d')

# 'start_time'과 'end_time'을 datetime으로 변환하여 'day'와 결합
data2['start_datetime'] = pd.to_datetime(data2['day'].astype(str) + ' ' + data2['start_time'].astype(str))
data2['end_datetime'] = pd.to_datetime(data2['day'].astype(str) + ' ' + data2['end_time'].astype(str))

# data2에서 시간 범위에 해당하는 행을 찾는 함수
def match_row(row, data2):
    matched = data2[
        (pd.to_datetime(row['Collect Date_Dam']) >= data2['start_datetime']) & 
        (pd.to_datetime(row['Collect Date_Dam']) <= data2['end_datetime'])
    ]
    return matched if not matched.empty else pd.Series([None]*len(data2.columns), index=data2.columns)

# data1에 대해 apply로 매칭 수행
matched_data = data1.apply(lambda row: match_row(row, data2), axis=1)

# 결과 데이터프레임 병합
result = pd.concat([data1, matched_data], axis=1)

In [58]:
data2['end_datetime']

0     2024-03-19 10:30:00
1     2024-03-19 12:00:00
2     2024-03-19 15:00:00
3     2024-03-19 18:00:00
4     2024-03-19 20:00:00
              ...        
100   2024-04-28 10:30:00
101   2024-04-28 12:00:00
102   2024-04-28 15:00:00
103   2024-04-28 18:00:00
104   2024-04-28 20:00:00
Name: end_datetime, Length: 105, dtype: datetime64[ns]

In [57]:
matched_data

0                  day start_time  end_time  Dam Thickn...
1        day                                         No...
2        day                                         No...
3        day                                         No...
4        day                                         No...
                               ...                        
40501    day                                         No...
40502              day start_time  end_time  Dam Thickn...
40503    day                                         No...
40504    day                                         No...
40505    day                                         No...
Length: 40506, dtype: object

In [15]:
data.train.get('Date')

,Collect Date_Dam,Collect Date_AutoClave,Collect Date_Fill1,Collect Date_Fill2
0,2024-04-25 11:10:00,2024-04-25 11:50:00,2024-04-25 11:20:00,2024-04-25 11:20:00
1,2023-09-19 14:30:00,2023-09-19 15:00:00,2023-09-19 14:30:00,2023-09-19 14:30:00
2,2024-03-05 09:30:00,2024-03-05 10:10:00,2024-03-05 09:30:00,2024-03-05 09:30:00
3,2023-09-25 15:40:00,2023-09-25 16:20:00,2023-09-25 15:40:00,2023-09-25 15:40:00
4,2023-06-27 13:20:00,2023-06-27 14:00:00,2023-06-27 13:20:00,2023-06-27 13:20:00
...,...,...,...,...
40501,2023-09-14 15:50:00,2023-09-14 16:30:00,2023-09-14 15:50:00,2023-09-14 15:50:00
40502,2024-04-10 14:20:00,2024-04-10 15:00:00,2024-04-10 14:20:00,2024-04-10 14:20:00
40503,2024-02-22 08:40:00,2024-02-22 09:20:00,2024-02-22 08:50:00,2024-02-22 08:50:00
40504,2023-07-25 11:00:00,2023-07-25 11:40:00,2023-07-25 11:00:00,2023-07-25 11:00:00
